**Preamble**

- This code cleans the raw public procurement data obtained from the State Audit Courts (TCEs) of the following states: CE, PE, MG, PR, RS and PB.
- The final output of this code is the tender table (_licitacao_), available at [basedosdados](https://basedosdados.org/dataset/d3874769-bcbd-4ece-a38a-157ba1021514?table=14c5d05b-9830-4710-b7ac-7e0ca1bf9d8b).
- Made by: Nathalia Sales


In [ ]:
import glob
import os
from zipfile import ZipFile

import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
# Connect to google drive
drive.mount("/content/gdrive")

# Display options

pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:.2f}".format

# Set directory

path = "/content/gdrive/MyDrive/ComprasPublicas_Brasil"

# Open some auxiliary files

municipio = pd.read_csv(
    os.path.join(path, "auxiliary_files/municipio.csv"),
    encoding="utf-8",
    dtype=str,
)

id_tce = pd.read_csv(
    os.path.join(path, "input/PE/municipios.csv"),
    encoding="latin-1",
    dtype=str,
    usecols=["CODIGOIBGE", "CODIGO", "UNIDADEFEDERATIVA"],
)

id_tce = id_tce.rename(
    columns={
        "CODIGOIBGE": "id_municipio",
        "CODIGO": "id_municipio_tce",
        "UNIDADEFEDERATIVA": "sigla_uf",
    },
)

# Merge both
municipio = municipio.merge(
    id_tce,
    how="left",
    left_on=["id_municipio", "sigla_uf"],
    right_on=["id_municipio", "sigla_uf"],
)

ug_id = pd.read_csv(
    os.path.join(path, "auxiliary_files/ug_id_mg.csv"), sep=",", dtype=str
)  # MG

orgao_municipio = pd.read_csv(
    os.path.join(path, "input/RS/orgaos_auditados_rs.csv"),
    encoding="utf-8",
    dtype=str,
    usecols=["CD_MUNICIPIO_IBGE", "CD_ORGAO"],
)  # RS

# Create a list of UFs
ufs = municipio["sigla_uf"].unique().tolist()

# Set columns order

ordem = [
    "ano",
    "mes",
    "sigla_uf",
    "id_municipio",
    "orgao",
    "id_unidade_gestora",
    "id_licitacao_bd",
    "id_licitacao",
    "id_dispensa",
    "ano_processo",
    "data_abertura",
    "data_edital",
    "data_homologacao",
    "data_publicacao_dispensa",
    "descricao_objeto",
    "natureza_objeto",
    "modalidade",
    "natureza_processo",
    "tipo",
    "forma_pagamento",
    "valor_orcamento",
    "valor",
    "valor_corrigido",
    "situacao",
    "estagio",
    "preferencia_micro_pequena",
    "exclusiva_micro_pequena",
    "contratacao",
    "quantidade_convidados",
    "tipo_cadastro",
    "carona",
    "covid_19",
]

## CE


In [ ]:
# CE

# Get a list of all CSV files

all_files = glob.glob(
    os.path.join(path, "input/CE/Licitações/licitacoes_*.csv")
)

# Initialize an empty list and loop through each CSV file

all_df_ce = []
for f in all_files:
    df1 = pd.read_csv(f, sep=";", dtype=str, encoding="latin-1")
    df1["arquivo"] = f.split("/")[-1]
    all_df_ce.append(df1)

# Concatenate all DataFrames in the list into a single DataFrame

ce = pd.concat(all_df_ce, ignore_index=True, sort=True)

# Extract the year from the file name

ce["ano"] = ce["arquivo"].str[11:15]

# List original variables to drop

ce_drop = [
    "cpf_gestor",
    "cpf_responsavel_homologacao",
    "cpf_responsavel_juridico",
    "data_criacao_comissao",
    "descricao1_justificativa_preco",
    "descricao1_motivo_fornecedor",
    "descricao2_justificativa_preco",
    "descricao2_motivo_fornecedor",
    "hora_licitacao",
    "arquivo",
    "nome_responsavel_homologacao",
    "nome_responsavel_juridico",
    "numero_comissao",
    "valor_limite_superior",
    "descricao2_objeto_licitacao",
    "nome_orgao_ata ",
    "data_realizacao_autuacao_licitacao",
]

# Dictionary

ce_rename = {
    "numero_licitacao": "id_licitacao",
    "descricao1_objeto_licitacao": "descricao_objeto",
    "modalidade_licitacao": "modalidade",
    "tipo_licitacao": "tipo",
    "valor_orcado_estimado": "valor_orcamento",
    "data_realizacao_licitacao": "data_abertura",
    "data_emissao_edital": "data_edital",
}

modalidade = {"4": "11", "5": "7", "6": "4", "7": "12"}

tipo = {"6": "4", "7": "5", "8": "2", "9": "13"}

# Drop and rename

ce = ce.drop(ce_drop, axis=1)
ce = ce.rename(ce_rename, axis=1)

# Read a CSV file containing municipality information

id_mun = pd.read_csv(
    os.path.join(path, "municipios.csv"),
    sep=";",
    dtype=str,
    encoding="latin-1",
    usecols=["geoibgeId", "codigo_municipio"],
)

id_mun = id_mun.rename({"geoibgeId": "id_municipio"}, axis=1)

# Merge on codigo_municipio to get id_municipio (IBGE code)

ce = ce.merge(
    id_mun,
    how="left",
    left_on="codigo_municipio",
    right_on="codigo_municipio",
)

# Replace by dictionary

ce["modalidade"] = ce["modalidade"].replace(modalidade, regex=True)
ce["tipo"] = ce["tipo"].replace(tipo, regex=True)

# Adjustments in type of purchase: dispensa and inexigibilidade
# Type does not apply but we know from other variable that it is tender waiver or non-requeriment
# 9 - modalidade não se aplica, D - dispensa, I - inexigibilidade

ce["modalidade"] = np.where(
    (ce["modalidade"] == "9")
    & (ce["modalidade_processo_administrativo"] == "D"),
    "8",
    ce["modalidade"],
)
ce["modalidade"] = np.where(
    (ce["modalidade"] == "9")
    & (ce["modalidade_processo_administrativo"] == "I"),
    "10",
    ce["modalidade"],
)

# Type does not apply and we don't know what it is - set as missing
# N - normal, R - registro de preços

ce["modalidade"] = np.where(
    (ce["modalidade"] == "9")
    & (ce["modalidade_processo_administrativo"] == "N"),
    np.nan,
    ce["modalidade"],
)
ce["modalidade"] = np.where(
    (ce["modalidade"] == "9")
    & (ce["modalidade_processo_administrativo"] == "R"),
    np.nan,
    ce["modalidade"],
)

# P - Regras de Organismos Internacionais
ce["modalidade"] = np.where(
    ce["modalidade_processo_administrativo"] == "P", "29", ce["modalidade"]
)

# Piggyback procurement

ce["carona"] = np.where(ce["modalidade_processo_administrativo"] == "R", 1, 0)

# Format date columns

ce["data_abertura"] = ce["data_abertura"].str[:10]
ce["data_edital"] = ce["data_edital"].str[:10]
ce["data_homologacao"] = ce["data_homologacao"].str[:10]

# Create valor_corrigido
# If both 'valor' and 'valor_orcamento' are not null, choose the smaller of the two values.
# If 'valor' is not null but 'valor_orcamento' is null, use 'valor'.
# If 'valor' is null but 'valor_orcamento' is not null, use 'valor_orcamento'
# In this case, we only have 'valor_orcamento', so they will be the same

ce["valor_orcamento"] = ce["valor_orcamento"].astype(float)
ce["valor"] = np.nan

ce["valor_corrigido"] = ce.apply(
    lambda x: (
        min(x["valor"], x["valor_orcamento"])
        if pd.notna(x["valor"]) and pd.notna(x["valor_orcamento"])
        else x["valor"]
        if pd.notna(x["valor"])
        else x["valor_orcamento"]
    ),
    axis=1,
)

# Assign state acronym to the 'sigla_uf'
ce["sigla_uf"] = "CE"

# Create a unique identifier for each purchase

ce["id_licitacao_bd"] = (
    ce["id_licitacao"]
    + ce["id_municipio"]
    + ce["ano"].str[2:4]
    + ce["sigla_uf"]
)

# String correction

ce["descricao_objeto"] = ce["descricao_objeto"].str.replace(
    "ADMINSITRATIVAS", "ADMINISTRATIVAS"
)

# Drop duplicates in all variables

ce = ce.drop_duplicates()

# Replace duplicated values in the 'id_licitacao_bd' column with missing

ce.loc[ce.duplicated(["id_licitacao_bd"], keep=False), "id_licitacao_bd"] = (
    np.nan
)

# print(ce.duplicated(subset=['id_licitacao_bd']).value_counts(normalize=True)*100)
# 0.87% missing

# Reorder columns
ce = ce.reindex(columns=ordem)

# Save
ce.to_csv(
    os.path.join(path, "output/licitacao_ce.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

## PE


In [ ]:
# PE

# Get a list of all CSV files

all_files = glob.glob(
    os.path.join(path, "input/PE/Licitações/licitacoesdetalhes_*.csv")
)

# Initialize an empty list and loop through each CSV file

all_df = []
for f in all_files:
    df1 = pd.read_csv(f, sep=",", encoding="latin-1", dtype=str)
    df1["arquivo"] = f.split("/")[-1]
    all_df.append(df1)

# Concatenate all DataFrames in the list into a single DataFrame

pe = pd.concat(all_df, ignore_index=True, sort=True)

# Extract the year from the file name

pe["ano"] = pe["arquivo"].str[19:23]

# List original variables to drop

pe_drop = [
    "ADJUDICADA",
    "ANOMODALIDADE",
    "CODIGODESCRICAOOBJETO",
    "CODIGOESTAGIOLICITACAO",
    "CODIGONATUREZA",
    "CODIGOOBJETO",
    "CODIGOSITUACAOLICITACAO",
    "OBJETOCONFORMEEDITAL",
    "RAZAOSOCIAL",
    "RESULTADOHABILITACAO",
    "UG",
    "ESPECIFICACAOOBJETO",
    "NUMERODOCUMENTOAJUSTADO",
    "NUMEROMODALIDADE",
    "NUMEROPROCESSO",
    "TOTALADJUDICADOLICITANTE",
    "LinkArquivo",
    "QTDELICITANTES",
    "FUNDAMENTOLEGAL",
    "DOTACAOORCAMENTARIA",
    "DATAPUBLICACAOHABILITACAO",
    "arquivo",
]

# Dictionary

pe_rename = {
    "CODIGOUG": "id_unidade_gestora",
    "CODIGOPL": "id_licitacao",
    "CARACTERISTICAOBJETO": "contratacao",
    "ANOPROCESSO": "ano_processo",
    "SITUACAOLICITACAO": "situacao",
    "TOTALADJUDICADOLICITACAO": "valor",
    "DESCRICAOOBJETO": "descricao_objeto",
    "VALORORCAMENTOESTIMATIVO": "valor_orcamento",
    "DATAEMISSAOEDITAL": "data_edital",
    "DATASESSAOABERTURA": "data_abertura",
    "DATAPUBLICACAOHOMOLOGACAO": "data_homologacao",
    "ESTAGIOLICITACAO": "estagio",
    "NOMENATUREZA": "natureza_objeto",
    "NOMEMODALIDADE": "modalidade",
}

natureza_objeto = {
    "Serviços de Engenharia": "1",
    "Obras": "1",
    "Concessão": "4",
    "Permissão": "5",
    "Alienação de Bens": "6",
    "Compras": "8",
    "Outros Serviços": "9",
    "Locação de Bens": "10",
}

situacao = {
    "Concluído": "1",
    "Em Andamento": "2",
    "Processo Revogado": "3",
    "Processo Deserto": "4",
    "Processo Fracassado": "5",
    "Processo Anulado": "6",
    "Processo Sustado por Determinação do TCE": "7",
    "Processo Suspenso por Determinação Judicial": "8",
    "Processo Suspenso por Decisão Administrativa": "9",
    "Edital Impugnado Aguardando Retificação": "10",
}

estagio = {
    "Processo Instaurado": "1",
    "Edital Publicado": "2",
    "Licitantes Cadastrados": "3",
    "Habilitação Concluída": "5",
    "Julgamento Concluído": "6",
    "Processo Adjudicado / Homologado / Ratificado": "7",
}

contratacao = {
    "Integral sem Itens com Percentual Proposto": "5",
    "Por Lotes com Percentual Proposto": "6",
    "Por Itens com Percentual Proposto": "7",
    "Integral com Itens com Percentual Proposto": "8",
    "Por Itens": "1",
    "Por Lotes": "2",
    "Integral sem Itens": "3",
    "Integral com Itens": "4",
}

# Drop and rename

pe = pe.drop(pe_drop, axis=1)
pe = pe.rename(pe_rename, axis=1)

# Read a CSV file containing municipality information

ug = pd.read_csv(
    os.path.join(path, "input/PE/unidadesjurisdicionadas.csv"),
    sep=",",
    encoding="latin-1",
    dtype=str,
)

ug_drop = [
    "CODIGOTCE",
    "ESFERA",
    "PODER",
    "UNIDADEFEDERATIVA",
    "NATUREZA",
    "TIPOPESSOAJURIDICA",
    "ORGAO",
    "MUNICIPIO",
    "SIGLA",
    "SITUACAO",
    "CNPJ",
]
ug_rename = {
    "CODIGOMUNICIPIO": "id_municipio_tce",
    "ID_UNIDADE_GESTORA": "id_unidade_gestora",
}

ug = ug.drop(ug_drop, axis=1)
ug = ug.rename(columns=ug_rename)

# Merge on id_unidade_gestora to get id_municipio_tce (TCE code)

pe = pe.merge(
    ug,
    how="left",
    left_on=["id_unidade_gestora"],
    right_on=["id_unidade_gestora"],
    indicator=True,
)

pe = pe.drop("_merge", axis=1)

# Merge on id_municipio_tce to get id_municipio (IBGE code)

pe = pe.merge(
    municipio,
    how="left",
    left_on="id_municipio_tce",
    right_on="id_municipio_tce",
)

pe = pe.drop(["nome", "id_municipio_6", "id_municipio_tce"], axis=1)

# Drop duplicates: suppliers level to bid level

pe = pe.drop_duplicates(
    subset=["id_licitacao", "id_municipio", "id_unidade_gestora", "ano"],
)

# Create a unique identifier for each purchase

pe["id_licitacao_bd"] = (
    pe["id_licitacao"] + pe["id_unidade_gestora"] + pe["sigla_uf"]
)

# Format date columns

pe["data_edital"] = pe["data_edital"].str[:10]
pe["data_abertura"] = pe["data_abertura"].str[:10]
pe["data_homologacao"] = pe["data_homologacao"].str[:10]

# Create valor_corrigido
# If both 'valor' and 'valor_orcamento' are not null, choose the smaller of the two values.
# If 'valor' is not null but 'valor_orcamento' is null, use 'valor'.
# If 'valor' is null but 'valor_orcamento' is not null, use 'valor_orcamento'

pe["valor"] = pe["valor"].astype(float)
pe["valor_orcamento"] = pe["valor_orcamento"].astype(float)

pe["valor_corrigido"] = pe.apply(
    lambda x: (
        min(x["valor"], x["valor_orcamento"])
        if pd.notna(x["valor"]) and pd.notna(x["valor_orcamento"])
        else x["valor"]
        if pd.notna(x["valor"])
        else x["valor_orcamento"]
    ),
    axis=1,
)

# Replace by dictionary

pe["natureza_objeto"] = pe["natureza_objeto"].replace(
    natureza_objeto, regex=True
)
pe["situacao"] = pe["situacao"].replace(situacao, regex=True)
pe["estagio"] = pe["estagio"].replace(estagio, regex=True)
pe["contratacao"] = pe["contratacao"].replace(contratacao, regex=True)

pe["modalidade"] = pe["modalidade"].replace(
    [
        "Convite",
        "Tomada de Preços",
        "Concorrência",
        "Pregão Presencial",
        "Lei 13.303/2016 - Pregão Presencial",
        "Pregão Eletrônico",
        "Lei 13.303/2016 - Pregão Eletrônico",
        "Leilão",
        "Dispensa",
        "Lei 13.303/2016 - Dispensa",
        "Inexigibilidade",
        "Lei 13.303/2016 - Inexigibilidade",
        "Regime Diferenciado de Contratações (RDC)",
        "Procedimento de Licitação Próprio",
    ],
    ["1", "2", "3", "5", "5", "6", "6", "7", "8", "8", "10", "10", "12", ""],
)

pe["modalidade"] = pe["modalidade"].replace(
    [
        "BIRD - Contratação Direta (CD)",
        "BIRD - Comparação de Preços (CP)",
        "BIRD - Seleção Baseada na Qualidade e Custo (SBQC)",
        "BIRD - Licitação Pública Internacional (ICB)",
        "BIRD - Consultor Individual (CI)",
        "BIRD - Licitação Pública Nacional (NCB)",
        "BIRD - Seleção Pelo Menor Custo (SMC)",
        "BIRD - Seleção Baseada nas Qualificações do Consultor (SQC)",
        "BIRD - Banco Internacional para Reconstrução e Desenvolvimento",
    ],
    ["13", "14", "15", "16", "17", "18", "19", "20", ""],
)

pe["modalidade"] = pe["modalidade"].replace(
    [
        "BID - Contratação Direta (CD)",
        "BID - Comparação de Preços (CP)",
        "BID - Seleção Baseada na Qualidade e Custo (SBQC)",
        "BID - Consultor Individual (CI)",
        "BID - Licitação Pública Nacional (LPN)",
        "BID - Licitação Pública Internacional (LPI)",
        "BID - Seleção Baseada no Menor Custo (SBMC)",
        "BID - Seleção Baseada nas Qualificações do Consultor (SQC)",
    ],
    ["21", "22", "23", "24", "25", "26", "27", "28"],
)

# Reorder columns
pe = pe.reindex(columns=ordem)

# Save
pe.to_csv(
    os.path.join(path, "output/licitacao_pe.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

## MG


In [ ]:
# List municipalities to loop

municipio_mg = municipio.query('sigla_uf=="MG"')
municipios_mg = municipio_mg["id_municipio"].tolist()

# Rename and drop variables from different files

# Competitive procurement - licitacao.csv

mg1_rename = {
    "dat_pub_edital": "data_edital",
    "seq_unidade": "id_unidade_gestora",
    "dsc_forma_pagamento": "forma_pagamento",
    "dsc_ind_exclusiva_Micro": "exclusiva_micro_pequena",
    "dsc_ind_pref_micro": "preferencia_micro_pequena",
    "seq_licitacao": "id_licitacao",
    "seq_orgao": "orgao",
    "dsc_modalidade": "modalidade",
    "dsc_nat_objeto": "natureza_objeto",
    "dsc_nat_processo": "natureza_processo",
    "dsc_objeto_licitacao": "descricao_objeto",
    "num_mes_referencia": "mes",
    "num_ano_referencia": "ano",
    "dsc_tipo_licitacao": "tipo",
    "dsc_tipo_cadastro": "tipo_cadastro",
    "num_convidados": "quantidade_convidados",
    "dat_abert_proc_adm": "data_abertura",
    "num_ano_processo": "ano_processo",
}

mg1_drop = [
    "dat_receb_prev_doc",
    "dat_veic_pub_001",
    "dat_veic_pub_002",
    "dsc_clausula_prorrog",
    "dsc_criterio_adjud",
    "num_versao_arq",
    "dat_conv_edital",
    "dsc_criterio_aceit",
    "vlr_rsp_nao_processado",
    "vlr_rsp_processado",
    "dsc_ind_reserv_micro",
    "dsc_ind_sub_contrato_micro",
    "num_exercicio_edital",
    "num_modalidade",
    "num_prazo_execucao",
    "num_processo",
    "dsc_processo_lote",
    "dsc_veic_pub_001",
    "dsc_veic_pub_002",
    "dsc_unid_medida",
    "dsc_regime_exec_obra",
]

# Resources from competitive procurement - recLicitacao.csv

mg2_rename = {
    "seq_licitacao": "id_licitacao",
    "seq_orgao": "orgao",
    "dsc_funcao": "funcao",
    "dsc_subfuncao": "subfuncao",
    "dsc_programa": "programa",
    "dsc_acao": "acao",
    "num_mes_referencia": "mes",
    "num_ano_referencia": "ano",
    "dsc_nat_despesa": "natureza_despesa",
    "vlr_recurso": "valor_orcamento",
}

mg2_drop = [
    "seq_rec_licitacao",
    "dsc_dotacao",
    "dsc_subacao",
    "dsc_fonte_recurso",
    "num_versao_arq",
]

# Itens from competitive procurement - homologLicitacao.csv
# Get values - quantity * unitary price

mg3_rename = {
    "seq_item_licitacao": "id_item",
    "seq_licitacao": "id_licitacao",
    "seq_orgao": "orgao",
    "num_ano_referencia": "ano",
    "vlr_unitario": "valor_unitario",
    "num_quant_item": "quantidade",
}

mg3_drop = [
    "seq_hom_licitacao",
    "num_mes_referencia",
    "num_versao_arq",
    "vlr_pct_desconto",
    "vlr_pct_tax_adm",
    "vlr_global",
    "num_doc_vencedor",
    "nom_vencedor",
    "num_versao_arq",
]

# Non-competitive procurement - dispensa.csv

mg4_rename = {
    "dat_pub_termo": "data_publicacao_dispensa",
    "seq_unidade": "id_unidade_gestora",
    "dsc_forma_pagamento": "forma_pagamento",
    "seq_dispensa": "id_dispensa",
    "seq_orgao": "orgao",
    "dsc_tipo_processo": "modalidade",
    "dsc_nat_objeto": "natureza_objeto",
    "dsc_objeto": "descricao_objeto",
    "num_mes_referencia": "mes",
    "num_ano_referencia": "ano",
    "dsc_tipo_cadastro": "tipo_cadastro",
    "dat_abertura": "data_abertura",
    "num_ano_processo": "ano_processo",
}

mg4_drop = [
    "dsc_ind_processo_lote",
    "dsc_justificativa",
    "dsc_razao",
    "dsc_veiculo_pub",
    "num_versao_arq",
    "vlr_rsp_proc",
    "num_processo",
    "vlr_rsp_nao_proc",
]

# Resources from non-competitive procurement - recDispensa.csv

mg5_rename = {
    "seq_dispensa": "id_dispensa",
    "seq_orgao": "orgao",
    "dsc_funcao": "funcao",
    "dsc_subfuncao": "subfuncao",
    "dsc_programa": "programa",
    "dsc_acao": "acao",
    "num_mes_referencia": "mes",
    "num_ano_referencia": "ano",
    "dsc_nat_despesa": "natureza_despesa",
    "vlr_recurso": "valor_orcamento",
}

mg5_drop = [
    "seq_rec_dispensa",
    "dsc_dotacao",
    "dsc_subacao",
    "dsc_fonte_recurso",
    "num_versao_arq",
]

# Itens from non-competitive procurement - fornDispensa.csv
# Get values - quantity * unitary price

mg6_rename = {
    "seq_item_dispensa": "id_item",
    "seq_dispensa": "id_dispensa",
    "seq_orgao": "orgao",
    "num_ano_referencia": "ano",
    "vlr_item": "valor_unitario",
    "num_quant_item": "quantidade",
}

mg6_drop = [
    "seq_forn_dispensa",
    "num_inscr_estadual",
    "dsc_sigla_uf",
    "num_mes_referencia",
    "num_certidao_inss",
    "dat_emi_cert_inss",
    "dat_emi_cert_fgts",
    "dat_val_cert_fgts",
    "dat_val_cert_inss",
    "num_cert_fgts",
    "num_cndt",
    "dat_emi_cndt",
    "dat_val_cndt",
    "num_doc_fornecedor",
    "dsc_nom_fornecedor",
    "num_versao_arq",
]

# Dictionary

modalidade = {
    "1 - CONVITE": "1",
    "1 - DISPENSA": "8",
    "2 - TOMADA DE PREÇOS": "2",
    "2 - INEXIGIBILIDADE": "10",
    "3 - INEXIGIBILIDADE POR CREDENCIAMENTO/CHAMADA PÚBLICA": "10",
    "3 - CONCORRÊNCIA": "3",
    "4 - CONCURSO": "11",
    "4 - DISPENSA POR CHAMADA PÚBLICA": "8",
    "5 - PREGÃO PRESENCIAL": "5",
    "6 - PREGÃO ELETRÔNICO": "6",
    "7 - LEILÃO": "7",
}

natureza_processo = {
    "1 - NORMAL": "1",
    "2 - REGISTRO DE PREÇOS": "2",
    "3 - CREDENCIAMENTO/ CHAMADA PÚBLICA": "3",
    "3 - CREDENCIAMENTO/CHAMADA PÚBLICA": "3",
}

natureza_objeto = {
    "1 - OBRAS E SERVIÇOS DE ENGENHARIA": "1",
    "2 - COMPRAS E OUTROS SERVIÇOS": "2",
    "3 - LOCAÇÃO DE IMÓVEIS": "3",
    "3 - LOCAÇÃO DE IMOVÉIS": "3",
    "4 - CONCESSÃO": "4",
    "5 - PERMISSÃO": "5",
    "6 - ALIENAÇÃO DE BENS": "6",
    "7 - COMPRAS PARA OBRAS E/OU SERVIÇOS DE ENGENHARIA": "7",
    " - ": "",
}

tipo = {
    "1 - MENOR PREÇO": "1",
    "2 - MELHOR TÉCNICA": "2",
    "3 - TÉCNICA E PREÇO": "3",
    "4 - MAIOR LANCE OU OFERTA": "4",
    " - ": "",
}

tipo_cadastro = {"1 - CADASTRO INICIAL": "1", "2 - RETIFICAÇÃO": "2"}

preferencia_micro_pequena = {"SIM": "1", "NÃO": "0"}

In [ ]:
# List years

anos_mg = ["2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

# MG folder
folder = os.path.join(path, "input/MG")

# Initialize an empty list and loop through each CSV file

all_df_mg = []
for a in anos_mg:
    for m in municipios_mg:
        zip_files = os.path.join(folder, f"{a}/licitacao_{a}.zip")
        if a in ["2014", "2015", "2016"]:
            arquivo1 = f"licitacao/{m}/{a}.{m}.licitacao.licitacao.csv"
            arquivo2 = f"licitacao/{m}/{a}.{m}.licitacao.regadesao.csv"
            arquivo3 = f"licitacao/{m}/{a}.{m}.licitacao.recLicitacao.csv"
            arquivo4 = f"licitacao/{m}/{a}.{m}.licitacao.homologLicitacao.csv"
            arquivo5 = f"licitacao/{m}/{a}.{m}.licitacao.dispensa.csv"
            arquivo6 = f"licitacao/{m}/{a}.{m}.licitacao.recDispensa.csv"
            arquivo7 = f"licitacao/{m}/{a}.{m}.licitacao.fornDispensa.csv"

        elif a == "2017":
            arquivo1 = f"{a}/licitacao/{m}/{a}.{m}.licitacao.licitacao.csv"
            arquivo2 = f"{a}/licitacao/{m}/{a}.{m}.licitacao.regadesao.csv"
            arquivo3 = f"{a}/licitacao/{m}/{a}.{m}.licitacao.recLicitacao.csv"
            arquivo4 = (
                f"{a}/licitacao/{m}/{a}.{m}.licitacao.homologLicitacao.csv"
            )
            arquivo5 = f"{a}/licitacao/{m}/{a}.{m}.licitacao.dispensa.csv"
            arquivo6 = f"{a}/licitacao/{m}/{a}.{m}.licitacao.recDispensa.csv"
            arquivo7 = f"{a}/licitacao/{m}/{a}.{m}.licitacao.fornDispensa.csv"
        else:
            arquivo1 = f"{a}.{m}.licitacao.licitacao.csv"
            arquivo2 = f"{a}.{m}.licitacao.regadesao.csv"
            arquivo3 = f"{a}.{m}.licitacao.recLicitacao.csv"
            arquivo4 = f"{a}.{m}.licitacao.homologLicitacao.csv"
            arquivo5 = f"{a}.{m}.licitacao.dispensa.csv"
            arquivo6 = f"{a}.{m}.licitacao.recDispensa.csv"
            arquivo7 = f"{a}.{m}.licitacao.fornDispensa.csv"

        with ZipFile(zip_files) as z, z.open(arquivo1) as f:
            # Competitive procurement
            mg1 = pd.read_csv(f, sep=";", encoding="utf-8", dtype=str)
            mg1["id_municipio"] = m
            mg1["sigla_uf"] = "MG"

            # Drop and rename
            mg1 = mg1.rename(mg1_rename, axis=1)
            mg1 = mg1.drop(mg1_drop, axis=1)

            # Format date columns
            mg1["data_edital"] = (
                mg1["data_edital"].str[6:]
                + "-"
                + mg1["data_edital"].str[3:5]
                + "-"
                + mg1["data_edital"].str[:2]
            )
            mg1["data_abertura"] = (
                mg1["data_abertura"].str[6:]
                + "-"
                + mg1["data_abertura"].str[3:5]
                + "-"
                + mg1["data_abertura"].str[:2]
            )

            # Create a unique identifier for each purchase
            mg1["id_licitacao_bd"] = (
                mg1["id_licitacao"]
                + mg1["id_unidade_gestora"]
                + mg1["sigla_uf"]
            )

        # Resources from competitive procurement
        with z.open(arquivo3) as f:
            mg2 = pd.read_csv(f, sep=";", encoding="utf-8", dtype=str)
            mg2["id_municipio"] = m

            # Drop and rename
            mg2 = mg2.rename(mg2_rename, axis=1)
            mg2 = mg2.drop(mg2_drop, axis=1)

            # Format value columns
            mg2["valor_orcamento"] = mg2["valor_orcamento"].replace(
                "################", np.nan
            )
            mg2["valor_orcamento"] = mg2["valor_orcamento"].astype(float)

            # Create a unique identifier for each purchase
            mg2 = (
                mg2.groupby(
                    ["id_municipio", "orgao", "ano", "mes", "id_licitacao"]
                )["valor_orcamento"]
                .sum()
                .to_frame("valor_orcamento")
                .reset_index()
            )

        # Itens from competitive procurement
        with z.open(arquivo4) as f:
            mg3 = pd.read_csv(f, sep=";", encoding="utf-8", dtype=str)
            mg3["id_municipio"] = m

            # Drop and rename
            mg3 = mg3.rename(columns=mg3_rename)
            mg3 = mg3.drop(mg3_drop, axis=1)

            # Format value columns
            mg3["quantidade"] = mg3["quantidade"].astype(float)
            mg3["valor_unitario"] = mg3["valor_unitario"].astype(float)

            # Create valor = unitary price * quantity
            mg3["valor"] = np.where(
                (mg3["valor_unitario"].notna()) | (mg3["quantidade"].notna()),
                mg3["quantidade"] * mg3["valor_unitario"],
                np.nan,
            )

            # Group by tender - id_licitacao
            mg3 = (
                mg3.groupby(["id_municipio", "orgao", "ano", "id_licitacao"])[
                    "valor"
                ]
                .sum()
                .to_frame("valor")
                .reset_index()
            )

        # Non competitive procurement (dispensa/inexibilidade)
        with z.open(arquivo5) as f:
            mg4 = pd.read_csv(f, sep=";", encoding="utf-8", dtype="string")
            mg4["id_municipio"] = m
            mg4["sigla_uf"] = "MG"

            # Drop and rename
            mg4 = mg4.rename(mg4_rename, axis=1)
            mg4 = mg4.drop(mg4_drop, axis=1)

            # Format date columns
            mg4["data_publicacao_dispensa"] = (
                mg4["data_publicacao_dispensa"].str[6:]
                + "-"
                + mg4["data_publicacao_dispensa"].str[3:5]
                + "-"
                + mg4["data_publicacao_dispensa"].str[:2]
            )
            mg4["data_abertura"] = (
                mg4["data_abertura"].str[6:]
                + "-"
                + mg4["data_abertura"].str[3:5]
                + "-"
                + mg4["data_abertura"].str[:2]
            )

            # Create a unique identifier for each purchase
            mg4["id_licitacao_bd"] = (
                mg4["id_dispensa"]
                + mg4["id_unidade_gestora"]
                + mg4["sigla_uf"]
            )

        # Resources from non-competitive procurement
        with z.open(arquivo6) as f:
            mg5 = pd.read_csv(f, sep=";", encoding="utf-8", dtype=str)
            mg5["id_municipio"] = m

            # Drop and rename
            mg5 = mg5.rename(mg5_rename, axis=1)
            mg5 = mg5.drop(mg5_drop, axis=1)

            # Format value columns
            mg5["valor_orcamento"] = mg5["valor_orcamento"].replace(
                "################", np.nan
            )
            mg5["valor_orcamento"] = mg5["valor_orcamento"].astype(float)

            # Group by tender - id_licitacao
            mg5 = (
                mg5.groupby(
                    ["id_municipio", "orgao", "ano", "mes", "id_dispensa"]
                )["valor_orcamento"]
                .sum()
                .to_frame("valor_orcamento")
                .reset_index()
            )

        # Itens from non-competitive procurement
        with z.open(arquivo7) as f:
            mg6 = pd.read_csv(f, sep=";", encoding="utf-8", dtype=str)
            mg6["id_municipio"] = m
            mg6 = mg6.rename(columns=mg6_rename)
            mg6 = mg6.drop(mg6_drop, axis=1)

            # Format value columns
            mg6["valor_unitario"] = mg6["valor_unitario"].astype(float)
            mg6["quantidade"] = mg6["quantidade"].astype(float)

            # Create valor = unitary price * quantity
            mg6["valor"] = np.where(
                (mg6["valor_unitario"].notna()) | (mg6["quantidade"].notna()),
                mg6["quantidade"] * mg6["valor_unitario"],
                np.nan,
            )

            # Group by tender - id_dispensa
            mg6 = (
                mg6.groupby(["id_municipio", "orgao", "ano", "id_dispensa"])[
                    "valor"
                ]
                .sum()
                .to_frame("valor")
                .reset_index()
            )

        # Merge competitive procurement files
        merge1 = mg1.merge(
            mg2,
            how="left",
            left_on=["id_municipio", "id_licitacao", "ano", "mes", "orgao"],
            right_on=["id_municipio", "id_licitacao", "ano", "mes", "orgao"],
        )

        merge1 = merge1.merge(
            mg3,
            how="left",
            left_on=["id_municipio", "id_licitacao", "ano", "orgao"],
            right_on=["id_municipio", "id_licitacao", "ano", "orgao"],
        )

        # Merge non competitive procurement files
        merge2 = mg4.merge(
            mg5,
            how="left",
            left_on=["id_municipio", "id_dispensa", "ano", "mes", "orgao"],
            right_on=["id_municipio", "id_dispensa", "ano", "mes", "orgao"],
        )

        merge2 = merge2.merge(
            mg6,
            how="left",
            left_on=["id_municipio", "id_dispensa", "ano", "orgao"],
            right_on=["id_municipio", "id_dispensa", "ano", "orgao"],
        )

        # Append competitive and non competitive
        mg = merge1.append([merge2], ignore_index=True)

        # Append all
        all_df_mg.append(mg)

# Concatenate all DataFrames in the list into a single DataFrame

mg = pd.concat(all_df_mg, ignore_index=True, sort=True)

# Create valor_corrigido
# If both 'valor' and 'valor_orcamento' are not null, choose the smaller of the two values.
# If 'valor' is not null but 'valor_orcamento' is null, use 'valor'.
# If 'valor' is null but 'valor_orcamento' is not null, use 'valor_orcamento'

mg["valor_corrigido"] = mg.apply(
    lambda x: (
        min(x["valor"], x["valor_orcamento"])
        if pd.notna(x["valor"]) and pd.notna(x["valor_orcamento"])
        else x["valor"]
        if pd.notna(x["valor"])
        else x["valor_orcamento"]
    ),
    axis=1,
)

floats = ["vlr_empenhado", "vlr_liquidado", "vlr_pago"]
mg[floats] = mg[floats].astype(float)

# Replace by dictionary

mg["modalidade"] = mg["modalidade"].replace(modalidade, regex=True)
mg["natureza_processo"] = mg["natureza_processo"].replace(
    natureza_processo, regex=True
)
mg["natureza_objeto"] = mg["natureza_objeto"].replace(
    natureza_objeto, regex=True
)
mg["tipo"] = mg["tipo"].replace(tipo, regex=True)
mg["tipo_cadastro"] = mg["tipo_cadastro"].replace(tipo_cadastro, regex=True)
mg["preferencia_micro_pequena"] = mg["preferencia_micro_pequena"].replace(
    preferencia_micro_pequena, regex=True
)
mg["exclusiva_micro_pequena"] = mg["exclusiva_micro_pequena"].replace(
    preferencia_micro_pequena, regex=True
)

# Reorder columns
mg = mg.reindex(columns=ordem)

# Save
mg.to_csv(
    os.path.join(path, "output/licitacao_mg.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

## PR


In [ ]:
# List variables to drop and rename

pr_drop = [
    "DataReferencia",
    "dsClausulaProrrogacao",
    "dsRegimeExecucaoLicitacao",
    "nmEntidade",
    "nrEditalOrigem",
    "nrLicitacao",
    "nranoEditalOrigem",
    "ultimoEnvioSIMAMNesteExercicio",
    "dtEnvio",
    "nmMunicipio",
]

pr_rename = {
    "cdIBGE": "id_municipio",
    "dsAvaliacaoLicitacao": "tipo",
    "dsModalidadeLicitacao": "modalidade",
    "dsNaturezaLicitacao": "natureza_processo",
    "idLicitacao": "id_licitacao",
    "dsObjeto": "descricao_objeto",
    "dsClassificacaoObjetoLicitacao": "natureza_objeto",
    "dsTipoSituacaoLicitacao": "situacao",
    "dtAbertura": "data_abertura",
    "dtEdital": "data_edital",
    "idPessoa": "id_unidade_gestora",
    "nrAnoLicitacao": "ano",
    "vlLicitacao": "valor_orcamento",
    "dtOcorrencia": "data_homologacao",
}

item_pr_rename = {
    "cdIBGE": "id_municipio",
    "idlicitacao": "id_licitacao",
    "idPessoa": "id_unidade_gestora",
    "nrAnoLicitacao": "ano",
    "nrQuantidadeVencedorLicitacao": "quantidade",
    "vlLicitacaoVencedorLicitacao": "valor_vencedor",
    "nrItem": "numero",
    "nrLote": "numero_lote",
    "nrClassificacao": "numero_classificacao",
}

# Dictionary

modalidade = {
    "Convite": "1",
    "Tomada de Preços": "2",
    "Concorrência": "3",
    "Pregão": "4",
    "Leilão": "7",
    "Processo Dispensa": "8",
    "Processo Inexigibilidade": "10",
    "Concurso": "11",
    "Regime Diferenciado de Contratações - RDC": "12",
    "Lei Ordinária nº 13.303/2016": "",
}

natureza_objeto = {
    "Obras e Serviços de Engenharia": "1",
    "Compras e Serviços": "2",
    "Alienação de Bens": "6",
    "Concessão de Direito Real de Uso": "11",
    "Cessão de Direitos": "12",
}

tipo = {
    "Menor Preço - Item": "1",
    "Menor Preço - Lote": "1",
    "Melhor Técnica - Item": "2",
    "Melhor Técnica - Lote": "2",
    "Técnica e Preço - Item": "3",
    "Técnica e Preço - Lote": "3",
    "Maior Lance ou Oferta - Item": "4",
    "Maior Lance ou Oferta - Lote": "4",
    "Maior Retorno Econômico": "5",
    "Dispensa/Inexigibilidade": "13",
}

situacao = {
    "Homologada": "1",
    "Andamento": "2",
    "Andamento – Nova Data de Abertura": "2",  # noqa: RUF001
    "Revogada": "3",
    "Deserta": "4",
    "Fracassada": "5",
    "Anulada": "6",
}

natureza_processo = {
    "Lei Ordinária nº 13.303/2016 - Dados Abertos, Lei Ordinária nº 13.303/2016 - Dados Sigilosos, Normal, ": "4",
    "Lei Ordinária nº 13.303/2016 - Dados Abertos, Lei Ordinária nº 13.303/2016 - Dados Sigilosos, Normal, Proc.de Disp.Inc I e II do art.24 da Lei 8666/93, ": "5",
    "Credenciamento, Lei Ordinária nº 13.303/2016 - Dados Abertos, Lei Ordinária nº 13.303/2016 - Dados Sigilosos, Normal, Registro de Preços, ": "6",
    "Normal, Registro de Preços, ": "6",
    "Lei Ordinária nº 13.303/2016 - Dados Abertos, Lei Ordinária nº 13.303/2016 - Dados Sigilosos, ": "",
}

In [ ]:
# List municipalities to loop
# Drop municipalities for which we couldn't transform xml in csv

municipio_pr = municipio.query('sigla_uf=="PR"')

municipio_pr = municipio_pr.query(
    'id_municipio_6 != "411915" & \
                                   id_municipio_6 != "411370" & \
                                   id_municipio_6 != "411535" & \
                                   id_municipio_6 != "411710" & \
                                   id_municipio_6 != "412627" & \
                                   id_municipio_6 != "410140" & \
                                   id_municipio_6 != "410350"'
)

municipios_pr = municipio_pr["id_municipio_6"].tolist()

anos_pr = [
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
    "2018",
    "2019",
    "2020",
    "2021",
]

# Initialize an empty list and loop through each CSV file

all_files_pr = []
for a in anos_pr:
    for m in municipios_pr:
        exec(
            f"path_lic = '/content/gdrive/MyDrive/ComprasPublicas_Brasil/input/PR/{a}/Licitacao/{m}/{a}_{m}_Licitacao.csv'"
        )

        pr = pd.read_csv(path_lic, sep=",", encoding="utf-8", dtype=str)  # noqa: F821

        # Drop and rename
        pr = pr.drop(pr_drop, axis=1)
        pr = pr.rename(pr_rename, axis=1)

        # Format date columns
        pr["data_edital"] = pr["data_edital"].str[:10]
        pr["data_abertura"] = pr["data_abertura"].str[:10]
        pr["data_homologacao"] = pr["data_homologacao"].str[:10]

        # Merge id_municipio 6 and 7 digits
        pr["id_municipio"] = pr["id_municipio"].astype(str)
        pr = pr.merge(
            municipio,
            how="left",
            left_on="id_municipio",
            right_on="id_municipio_6",
        )

        pr = pr.drop(["id_municipio_x", "id_municipio_6", "nome"], axis=1)
        pr = pr.rename({"id_municipio_y": "id_municipio"}, axis=1)

        # Create a unique identifier for each purchase
        pr["id_licitacao_bd"] = (
            pr["id_licitacao"] + pr["id_unidade_gestora"] + pr["sigla_uf"]
        )

        # Replace by dictionary

        pr["modalidade"] = pr["modalidade"].replace(modalidade)
        pr["natureza_objeto"] = pr["natureza_objeto"].replace(natureza_objeto)
        pr["tipo"] = pr["tipo"].replace(tipo)
        pr["situacao"] = pr["situacao"].replace(situacao)
        pr["natureza_processo"] = pr["natureza_processo"].replace(
            natureza_processo
        )

        # Merge with files LicitacaoVencedor to get the variable 'valor'
        # Goal: aggregate unitary price * quantity by tender

        exec(
            f"path_lic_venc = '/content/gdrive/MyDrive/ComprasPublicas_Brasil/input/PR/{a}/Licitacao/{m}/{a}_{m}_LicitacaoVencedor.csv'"
        )

        item_pr = pd.read_csv(
            path_lic_venc,  # noqa: F821
            sep=",",
            encoding="utf-8",
            dtype=str,
            usecols=[
                "nrAnoLicitacao",
                "cdIBGE",
                "idlicitacao",
                "idPessoa",
                "nrQuantidadeVencedorLicitacao",
                "vlLicitacaoVencedorLicitacao",
                "nrLote",
                "nrItem",
                "nrClassificacao",
            ],
        )

        item_pr = item_pr.rename(item_pr_rename, axis=1)

        # Merge id_municipio 6 and 7 digits
        item_pr = item_pr.merge(
            municipio,
            how="left",
            left_on="id_municipio",
            right_on="id_municipio_6",
        )

        item_pr = item_pr.drop(
            ["id_municipio_x", "id_municipio_6", "nome"], axis=1
        )
        item_pr = item_pr.rename({"id_municipio_y": "id_municipio"}, axis=1)

        # Drop non suppliers
        item_pr["min_classificacao"] = item_pr.groupby(
            ["ano", "id_municipio", "id_licitacao", "numero_lote", "numero"]
        )["numero_classificacao"].transform("min")
        item_pr["diff"] = np.where(
            item_pr["numero_classificacao"] == item_pr["min_classificacao"],
            0,
            1,
        )
        item_pr = item_pr[item_pr["diff"] == 0]

        # Create a unique identifier for each purchase
        item_pr["id_licitacao_bd"] = (
            item_pr["id_licitacao"]
            + item_pr["id_unidade_gestora"]
            + item_pr["sigla_uf"]
        )

        # Format value columns
        item_pr["valor_vencedor"] = item_pr["valor_vencedor"].astype(float)
        item_pr["quantidade"] = item_pr["quantidade"].astype(float)

        # Create valor = unitary price * quantity
        item_pr["valor_total"] = np.where(
            (item_pr["quantidade"].isna())
            | (item_pr["valor_vencedor"].isna()),
            np.nan,
            item_pr["quantidade"] * item_pr["valor_vencedor"],
        )

        item_pr["valor_total"] = item_pr["valor_total"].astype(float)

        # Group by tender (id_licitacao_bd)
        valor_total = (
            item_pr.groupby(["ano", "id_municipio", "id_licitacao_bd"])[
                "valor_vencedor", "valor_total"
            ]
            .sum()
            .reset_index()
        )

        # Merge with tender dataframe
        pr = pr.merge(
            valor_total,
            how="left",
            left_on=["ano", "id_municipio", "id_licitacao_bd"],
            right_on=["ano", "id_municipio", "id_licitacao_bd"],
        )

        # Append all
        all_files_pr.append(pr)

# Concatenate all DataFrames in the list into a single DataFrame

pr = pd.concat(all_files_pr, ignore_index=True, sort=True)

# Deal with outliers

pr["valor_orcamento"] = pr["valor_orcamento"].astype(float)
pr["percentile"] = pr["valor_orcamento"].quantile(0.999)
pr["valor_orcamento"] = np.where(
    pr["valor_orcamento"] > pr["percentile"], np.nan, pr["valor_orcamento"]
)

pr = pr.rename({"valor_total": "valor"}, axis=1)
pr["percentile2"] = pr["valor"].quantile(0.999)
pr["valor"] = np.where(pr["valor"] > pr["percentile2"], np.nan, pr["valor"])

# Create valor_corrigido
# If both 'valor' and 'valor_orcamento' are not null, choose the smaller of the two values.
# If 'valor' is not null but 'valor_orcamento' is null, use 'valor'.
# If 'valor' is null but 'valor_orcamento' is not null, use 'valor_orcamento'

pr["valor_corrigido"] = pr.apply(
    lambda x: (
        min(x["valor"], x["valor_orcamento"])
        if pd.notna(x["valor"]) and pd.notna(x["valor_orcamento"])
        else x["valor"]
        if pd.notna(x["valor"])
        else x["valor_orcamento"]
    ),
    axis=1,
)
# Reorder columns
pr = pr.reindex(columns=ordem)

# Save
pr.to_csv(
    os.path.join(path, "output/licitacao_pr.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

## RS


In [ ]:
# List municipalities to loop

municipio_rs = municipio.query('sigla_uf=="RS"')
municipios_rs = municipio_rs["id_municipio"].tolist()

# List variables to drop and rename

rs_rename = {
    "CD_ORGAO": "orgao",
    "ANO_LICITACAO": "ano",
    "CD_TIPO_MODALIDADE": "modalidade",
    "TP_OBJETO": "natureza_objeto",
    "TP_LICITACAO": "tipo",
    "TP_CARACTERISTICA_OBJETO": "contratacao",
    "TP_NATUREZA": "natureza_processo",
    "TP_RESULTADO_GLOBAL": "situacao",
    "DS_OBJETO": "descricao_objeto",
    "VL_LICITACAO": "valor_orcamento",
    "DT_ABERTURA": "data_abertura",
    "DT_HOMOLOGACAO": "data_homologacao",
    "VL_HOMOLOGADO": "valor",
    "BL_COVID19": "covid_19",
    "CD_TIPO_FASE_ATUAL": "estagio",
    "CD_MUNICIPIO_IBGE": "id_municipio",
    "NR_LICITACAO": "id_licitacao",
    "ANO_PROCESSO": "ano_processo",
}

rs_drop = [
    "NR_COMISSAO",
    "ANO_COMISSAO",
    "TP_COMISSAO",
    "NR_PROCESSO",
    "TP_NIVEL_JULGAMENTO",
    "DT_AUTORIZACAO_ADESAO",
    "TP_REGIME_EXECUCAO",
    "BL_PERMITE_SUBCONTRATACAO",
    "TP_FORNECIMENTO",
    "TP_ATUACAO_REGISTRO",
    "NR_LICITACAO_ORIGINAL",
    "ANO_LICITACAO_ORIGINAL",
    "NR_ATA_REGISTRO_PRECO",
    "DT_ATA_REGISTRO_PRECO",
    "PC_TAXA_RISCO",
    "TP_EXECUCAO",
    "TP_DISPUTA",
    "TP_PREQUALIFICACAO",
    "BL_INVERSAO_FASES",
    "CNPJ_ORGAO_GERENCIADOR",
    "NM_ORGAO_GERENCIADOR",
    "CD_TIPO_FUNDAMENTACAO",
    "NR_ARTIGO",
    "DS_INCISO",
    "DS_LEI",
    "DT_INICIO_INSCR_CRED",
    "DT_FIM_INSCR_CRED",
    "DT_INICIO_VIGEN_CRED",
    "DT_FIM_VIGEN_CRED",
    "BL_ORCAMENTO_SIGILOSO",
    "BL_RECEBE_INSCRICAO_PER_VIG",
    "BL_PERMITE_CONSORCIO",
    "BL_LICIT_PROPRIA_ORGAO",
    "TP_DOCUMENTO_FORNECEDOR",
    "NR_DOCUMENTO_FORNECEDOR",
    "TP_DOCUMENTO_VENCEDOR",
    "NR_DOCUMENTO_VENCEDOR",
    "BL_GERA_DESPESA",
    "DS_OBSERVACAO",
    "PC_TX_ESTIMADA",
    "PC_TX_HOMOLOGADA",
    "DT_ADJUDICACAO",
    "BL_COMPARTILHADA",
    "LINK_LICITACON_CIDADAO",
]

# Dictionary

modalidade = {
    "CNV": "1",
    "TMP": "2",
    "CNC": "3",
    "CCP": "3",
    "CCE": "3",
    "PRP": "5",
    "PCP": "5",
    "PRE": "6",
    "PCE": "6",
    "LEI": "7",
    "LEE": "7",
    "PRD": "8",
    "CPP": "8",
    "PDE": "9",
    "PRI": "10",
    "CNS": "11",
    "RDC": "12",
    "RDE": "12",
    "RIN": "29",
    "CPC": "30",
    "CHP": "30",
    "RPO": "",
    "ESE": "",
    "EST": "",
}

contratacao = {"IT": "1", "IU": "1", "LT": "2", "LU": "2"}

tipo = {
    "MPR": "1",
    "MDE": "1",
    "MTC": "2",
    "MPP": "2",
    "TPR": "3",
    "MLO": "4",
    "MOP": "4",
    "MRE": "5",
    "MCA": "6",
    "MOO": "7",
    "MOQ": "7",
    "MOT": "8",
    "MVT": "9",
    "MTO": "10",
    "MTT": "11",
    "MTX": "12",
    "NSA": "13",
}

estagio = {"INT": "1", "EPU": "2", "PUB": "2", "HAP": "4", "ADH": "6"}

situacao = {"A": "1", "R": "3", "D": "4", "F": "5", "N": "6"}

natureza_objeto = {
    "OSE": "1",
    "CSE": "2",
    "LOC": "3",
    "COL": "4",
    "PER": "5",
    "ALB": "6",
    "COM": "8",
    "OUS": "9",
    "CON": "11",
    "PRI": "13",
    "PPP": "14",
    "SAU": "15",
    "INF": "16",
}

natureza_processo = {"N": "1", "R": "2", "O": "3"}

covid_19 = {"S": "1", "N": "0"}

In [ ]:
# licitacao.csv

# RS folder
folder = os.path.join(path, "input/RS")

anos_rs = ["2016", "2017", "2018", "2019", "2020", "2021"]

# Initialize an empty list and loop through each CSV file

all_df_rs = []
for a in anos_rs:
    df = os.path.join(
        folder,
        f"Licitacao/{a}.csv.zip",
    )
    with ZipFile(df) as z, z.open("licitacao.csv") as f:
        rs = pd.read_csv(f, sep=",", encoding="utf-8", dtype=str)

        rs = rs.drop(rs_drop, axis=1)

        # Merge to get id_municipio
        rs = rs.merge(
            orgao_municipio,
            how="left",
            left_on="CD_ORGAO",
            right_on="CD_ORGAO",
            indicator=True,
        )  # alguns ids missing

        rs = rs[rs["_merge"] == "both"]

        rs = rs.drop(
            [
                "NOME_ORGAO",
                "ESFERA",
                "SIGLA_ORGAO",
                "SETOR_GOVERNAMENTAL",
                "CNPJ",
                "NATUREZA_JURIDICA",
                "CONTABILIDADE",
                "SITUACAO_ORGAO",
                "CD_MUNICIPIO_TCERS",
                "NOME_MUNICIPIO",
            ],
            axis=1,
        )

        rs = rs.rename(rs_rename, axis=1)

        rs["exclusiva_micro_pequena"] = np.where(
            rs["TP_BENEFICIO_MICRO_EPP"] == "L", 1, 0
        )  # L - licitacao exclusiva
        rs["preferencia_micro_pequena"] = np.where(
            (rs["TP_BENEFICIO_MICRO_EPP"] == "T")
            | (rs["TP_BENEFICIO_MICRO_EPP"] == "S"),
            1,
            0,
        )  # S - tratamento diferenciado ou subcontratacao

        rs = rs.drop("TP_BENEFICIO_MICRO_EPP", axis=1)

        rs = rs.drop(rs[(rs["modalidade"] == "MAI")].index)

        # Piggyback procurement
        # RPO - adesao a ata de registro de preco

        rs["carona"] = np.where(rs["modalidade"] == "RPO", 1, 0)

        rs["sigla_uf"] = "RS"

        # Create a unique identifier for each purchase
        rs["id_licitacao_bd"] = (
            rs["id_licitacao"]
            + rs["ano"]
            + rs["modalidade"]
            + rs["orgao"]
            + rs["sigla_uf"]
        )

        # Replace by dictionary
        rs["modalidade"] = rs["modalidade"].replace(modalidade, regex=True)
        rs["contratacao"] = rs["contratacao"].replace(contratacao, regex=True)
        rs["tipo"] = rs["tipo"].replace(tipo, regex=True)
        rs["estagio"] = rs["estagio"].replace(estagio, regex=True)
        rs["situacao"] = rs["situacao"].replace(situacao, regex=True)
        rs["natureza_objeto"] = rs["natureza_objeto"].replace(
            natureza_objeto, regex=True
        )
        rs["natureza_processo"] = rs["natureza_processo"].replace(
            natureza_processo, regex=True
        )
        rs["covid_19"] = rs["covid_19"].replace(covid_19, regex=True)

        # Format value columns
        rs["valor_orcamento"] = rs["valor_orcamento"].astype(float)
        rs["valor"] = rs["valor"].replace("#################", np.nan)
        rs["valor"] = rs["valor"].astype(float)

        # Create valor_corrigido
        # If both 'valor' and 'valor_orcamento' are not null, choose the smaller of the two values.
        # If 'valor' is not null but 'valor_orcamento' is null, use 'valor'.
        # If 'valor' is null but 'valor_orcamento' is not null, use 'valor_orcamento'

        rs["valor_corrigido"] = rs.apply(
            lambda x: (
                min(x["valor"], x["valor_orcamento"])
                if pd.notna(x["valor"]) and pd.notna(x["valor_orcamento"])
                else x["valor"]
                if pd.notna(x["valor"])
                else x["valor_orcamento"]
            ),
            axis=1,
        )
    all_df_rs.append(rs)

# Concatenate all DataFrames in the list into a single DataFrame
rs = pd.concat(all_df_rs, ignore_index=True, sort=True)

# Reorder columns
rs = rs.reindex(columns=ordem)

# rs.query('ano!=ano_processo')
# 11629 obs

# Save
rs.to_csv(
    os.path.join(path, "output/licitacao_rs.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

## PB


In [ ]:
# PB folder
folder = os.path.join(path, "input/PB")

# Open files

pb = pd.read_csv(
    os.path.join(folder, "TCE-PB-Portal-Gestor-Licitacoes_Propostas.txt"),
    sep="|",
    encoding="utf-8",
    dtype=str,
)

pb_drop = [
    "jurisdicionado_id",
    "nome_jurisdicionado",
    "nome_tipo_jurisdicionado",
    "nome_tipo_administracao_jurisdicionado",
    "nome_esfera_jurisdicionado",
    "nome_setor_atual_licitacao",
    "nome_proponente",
    "cpf_cnpj_proponente",
    "valor_proposta",
    "situacao_proposta",
    "nome_estagio_processual_licitacao",
]

pb_rename = {
    "data_homologacao_licitacao": "data_homologacao",
    "cd_ugestora": "id_unidade_gestora",
    "objeto_licitacao": "descricao_objeto",
    "nome_modalidade_licitacao": "modalidade",
    "valor_estimado_licitacao": "valor_orcamento",
    "valor_licitado_licitacao": "valor",
    "situacao_fracassada_licitacao": "situacao",
    "protocolo_licitacao": "id_licitacao",
}

modalidade = {
    "Convite": "1",
    "Tomada de Preços": "2",
    "Concorrência": "3",
    "Pregão Medida Provisória 1.047/21": "4",
    "Pregão Lei 14.133/21": "4",
    "Pregão Presencial": "5",
    "Pregão Eletrônico": "6",
    "Leilão": "7",
    "Dispensa Art. 24 - Lei 8.666/93": "8",
    "Dispensa COVID-19 Art. 4º da Lei 13.979/2020": "8",
    "Dispensa Lei 14.133/21": "8",
    "Dispensa Medida Provisória 1.047/21": "8",
    "Dispensada Art. 17 - Lei 8.666/93": "8",
    "Inexigibilidade": "10",
    "10 Lei 14.133/21": "10",
    "Concurso": "11",
    "RDC - Regime Diferenciado de Contratações Públicas": "12",
    "Licitação Internacional GN 2350-9": "29",
    "Licitação Internacional GN 2349-9": "29",
    "Licitação Internacional Não Competitiva": "29",
    "Chamada Pública": "31",
    "Licitação da Lei Nº 13.303/2016": "",
    "Adesão a Ata de Registro de Preços": "",
    "10 Lei 14.133/21": "",  # noqa: F601
    " Art. 29 ou 30": "",
    "Contratação Emergencial de Organização Art. 12, II da Lei Nº 9.454/2011": "",
}

situacao = {"Não": "1", "Sim": "5", "GUARDA TEMPORÁRIA": ""}

pb = pb.rename(pb_rename, axis=1)

# nome_municipio - 50298 missing values, consórcios intermunicipais regionais: ex - Consórcio Intermunicipal de Saúde dos Municípios do Alto Sertão Paraibano ou esfera estadual
# print(pb['nome_municipio'].isna().sum())
pb = pb.dropna(subset=["nome_municipio"])

# Create year from numero_licitacao
pb["ano"] = pb["numero_licitacao"].str[6:10]
pb["ano"] = pb["ano"].str.replace("2104", "2014")
pb["ano"] = pb["ano"].str.replace("3014", "2014")

# Keep one row per tender
pb = pb[pb["situacao_proposta"] == "Vencedora"]
pb["sigla_uf"] = "PB"

pb = pb.drop(pb_drop, axis=1)

# Merge to get id_municipio
pb["nome_municipio"] = pb["nome_municipio"].str.replace(
    "Santa Terezinha", "Santa Teresinha"
)
pb["nome_municipio"] = pb["nome_municipio"].str.replace("Quixaba", "Quixabá")

pb = pb.merge(
    municipio,
    how="left",
    left_on=["nome_municipio", "sigla_uf"],
    right_on=["nome", "sigla_uf"],
    indicator=True,
)
pb = pb.drop(
    ["nome", "nome_municipio", "id_municipio_6", "id_municipio_tce"],
    axis=1,
)

# Format
pb["id_licitacao"] = pb["id_licitacao"].str[5:]
pb["id_licitacao"] = pb["id_licitacao"].str.replace("/", "")
pb["valor"] = pb["valor"].astype(float)
pb["valor_orcamento"] = pb["valor_orcamento"].astype(float)

# Create a unique identifier for each purchase
pb["id_licitacao_bd"] = (
    pb["id_licitacao"] + pb["id_unidade_gestora"] + pb["sigla_uf"]
)

# Piggyback procurement (adesao a ata de registro de preco)
pb["carona"] = np.where(
    pb["modalidade"] == "Adesão a Ata de Registro de Preços", 1, 0
)

# Covid 19 indicator
pb["covid_19"] = np.where(
    (pb["modalidade"] == "Dispensa COVID-19 (Art. 4º da Lei 13.979/2020)")
    | (pb["modalidade"] == "Dispensa Medida Provisória 1.047/21")
    | (pb["modalidade"] == "Pregão Medida Provisória 1.047/21"),
    1,
    0,
)

# Create valor_corrigido
# If both 'valor' and 'valor_orcamento' are not null, choose the smaller of the two values.
# If 'valor' is not null but 'valor_orcamento' is null, use 'valor'.
# If 'valor' is null but 'valor_orcamento' is not null, use 'valor_orcamento'

pb["valor_corrigido"] = pb.apply(
    lambda x: (
        min(x["valor"], x["valor_orcamento"])
        if pd.notna(x["valor"]) and pd.notna(x["valor_orcamento"])
        else x["valor"]
        if pd.notna(x["valor"])
        else x["valor_orcamento"]
    ),
    axis=1,
)

# Replace by dictionary
pb["modalidade"] = pb["modalidade"].str.replace("(", "")
pb["modalidade"] = pb["modalidade"].str.replace(")", "")
pb["modalidade"] = pb["modalidade"].str.rstrip()
pb["modalidade"] = pb["modalidade"].str.lstrip()
pb["modalidade"] = pb["modalidade"].replace(modalidade, regex=True)
pb["modalidade"] = pb["modalidade"].replace(
    "10 Lei 14.133/21", "10"
)  # force this one
pb["situacao"] = pb["situacao"].replace(situacao, regex=True)

# Format date
pb["data_homologacao"] = (
    pb["data_homologacao"].str[6:]
    + "-"
    + pb["data_homologacao"].str[3:5]
    + "-"
    + pb["data_homologacao"].str[:2]
)

# Drop duplicates: suppliers level to tender level
pb = pb.drop_duplicates(
    subset=["id_licitacao", "id_municipio", "id_unidade_gestora", "ano"],
)

# Drop years after 2021
pb["ano"] = pb["ano"].astype(int)
pb = pb[pb["ano"] < 2022]

# ~ 4593 obs onde ano ! = ano_homologacao

# Reorder columns
pb = pb.reindex(columns=ordem)

# Save
pb.to_csv(
    os.path.join(path, "output/licitacao_pb.csv"),
    index=False,
    na_rep="",
    float_format="%.2f",
)

## Partition


In [ ]:
# List of UFs
ufs = ["CE", "PE", "MG", "PR", "RS", "PB"]

# Loop over each UF
for uf in ufs:
    # Load the corresponding CSV file for the UF
    file_path = f"/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao_{uf.lower()}.csv"
    df = pd.read_csv(file_path, dtype=str, encoding="utf-8")

    # Convert 'ano' column to integer
    df["ano"] = df["ano"].astype(int)

    # Save cvs by year and state

    for ano in [*range(2012, 2022)]:
        for uf in ufs:
            if uf == "CE" and ano in [*range(2009, 2022)]:
                print(f"Particionando {ano} do CE")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao/ano={ano}/sigla_uf=CE/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "PE" and ano in [*range(2012, 2022)]:
                print(f"Particionando {ano} do PE")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao/ano={ano}/sigla_uf=PE/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "MG" and ano in [*range(2014, 2022)]:
                print(f"Particionando {ano} do MG")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao/ano={ano}/sigla_uf=MG/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "PR" and ano in [*range(2013, 2022)]:
                print(f"Particionando {ano} do PR")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao/ano={ano}/sigla_uf=PR/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "RS" and ano in [*range(2016, 2022)]:
                print(f"Particionando {ano} do RS")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao/ano={ano}/sigla_uf=RS/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )
            if uf == "PB" and ano in [*range(2014, 2022)]:
                print(f"Particionando {ano} do PB")
                df2 = df[df["ano"] == ano]
                df2 = df2.drop(["ano", "sigla_uf"], axis=1)
                exec(
                    f"df2.to_csv('/content/gdrive/MyDrive/ComprasPublicas_Brasil/output/licitacao/ano={ano}/sigla_uf=PB/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.2f')"
                )